In [2]:
import pandas as pd
import powerplantmatching as pm
import pypsa
import pycountry
import os 
import numpy as np

In [3]:
def prepare_capacity_table(capacity):
    
    capacity_matching = pd.Series(
        ["nuclear", "lignite", "coal", "gas", "oil", "other", "ROR", "ROR", "reservoir", "reservoir", "PHS",
         "reservoir (pumping)", "PHS (pumping)", "onwind", "offwind", "CSP", "solar", "biomass", "biomass", "battery"],
        capacity.index
    )

    capacity = capacity.groupby(capacity_matching, ).sum()
    
    pm_plants = pm.powerplants()
    gas_share = pd.DataFrame()
    gas_share["CCGT"] = pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", "CCGT",:].Capacity.div(
        pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", :].Capacity.groupby(level=1).sum(),
        fill_value=0
    )

    gas_share.index = [pycountry.countries.lookup(i).alpha_2 for i in gas_share.index]

    gas_share["OCGT"] = 1- gas_share["CCGT"]

    gas_share = gas_share.reindex(capacity.columns.str[:2]).fillna(gas_share.mean())
    gas_share.index = capacity.columns

    capacity = pd.concat([
        capacity.drop("gas"),
        gas_share.multiply(capacity.loc["gas"],axis=0).T
    ])
    
    return capacity


In [4]:
def build_capacity_table():

    capacity_years = pd.DataFrame()

    for sheet in plant_sheets.index:

        capacity_raw = pd.read_excel(excel_file, sheet, header=1, index_col=1).dropna(how="all", axis=1).iloc[:20]

        capacity_years[sheet[2:]] = prepare_capacity_table(capacity_raw).stack().astype(float)

    capacity_years.columns = capacity_years.columns.astype(int)
    capacity_years = capacity_years.reindex(years,axis=1)
    capacity_years.loc[distributed_resources, :] = capacity_years.loc[distributed_resources, :].interpolate(axis=1)
    

    return capacity_years.fillna(method="ffill", axis=1)[year].unstack(1)

In [5]:
def build_thermal_properties(properties):
    
    for col in properties.columns:
        try: 
            properties[col] = properties[col].astype(float)
        except:
            None

    properties.set_index(properties.index.remove_unused_levels(), inplace=True)

    properties_matching = pd.Series(
        ["OCGT", "coal", "oil", "oil", "lignite", "nuclear", "oil"],
        properties.index.levels[0]
    )

    properties_matching = properties_matching.reindex([i[0] for i in properties.index])

    properties_matching.index = properties.index

    properties_matching.loc[[i for i in properties_matching.index if "CCGT" in i[1]]] = "CCGT"

    properties_transformed = properties.iloc[:, 1:].groupby(
        properties_matching.reindex(properties.index).values
    ).mean()

    missing_properties = properties.reindex([("Gas", "conventional old 2"), ("Gas", "conventional old 2")]).copy()
    missing_properties.index = ["biomass", "other"]
    missing_properties.loc["biomass","CO2 emission factor"] = 0

    properties_transformed = pd.concat([properties_transformed, missing_properties[properties_transformed.columns]])

    properties_transformed.set_index(pd.Series("existing", properties_transformed.index),append=True, inplace=True)

    properties_new = properties.loc["Gas", ["CCGT new", "OCGT new"], :].set_index(pd.MultiIndex.from_product([["CCGT", "OCGT"], ["new"]]))

    properties_transformed = pd.concat([properties_transformed, properties_new[properties_transformed.columns]])
    properties_transformed.index.names=["carrier", "invest_status"]

    return properties_transformed

In [6]:
def build_inflows(inflows):
    base_year_hydro = pd.Series(
        inflows.index.levels[4].astype(int), 
        inflows.index.levels[4]).subtract(year).abs().idxmin()

    inflows = inflows.loc[:, str(climate_year), :, :, base_year_hydro]

    inflow_grouper = pd.Series(["ROR", "PHS", "reservoir", "reservoir", "ROR"], inflows.index.levels[1])

    inflows = inflows.unstack(1).groupby(inflow_grouper, axis=1).sum().multiply(1e3) # conversion to MWh
    
    inflows = inflows.unstack(0).stack(0)
    inflows.index = [" ".join(i) for i in inflows.index]
    
    inflows = inflows.T
    
    inflows.index = snapshots
    
    return inflows

In [7]:
def add_existing_storage():  
 
    storage = capacity.loc[["PHS", "reservoir", "battery"]].unstack().reset_index().copy()
    storage.columns = ["bus", "carrier", "p_nom"]
    storage.index = storage.bus + " " + storage.carrier

    p_min_pu = capacity.loc[["PHS (pumping)", "reservoir (pumping)",]].div(capacity.loc[["PHS", "reservoir"]].add(1e-5).values)
    p_min_pu.index = ["PHS", "reservoir"]

    storage["p_min_pu"] = (
        p_min_pu.unstack()
        .reindex(pd.MultiIndex.from_arrays([storage.bus, storage.carrier]))
    ).values

    storage.loc[storage.carrier=="battery", "p_min_pu"] = -1

    storage = storage[storage.p_nom >0]

    storage_inflows = inflows[storage.loc[storage.carrier=="reservoir"].index]

    storage_capacity_raw = pd.read_excel(excel_file, sheet, index_col=1, header=28).dropna(how="all", axis=1)

    storage_capacity= storage_capacity_raw.groupby(
        ["ROR", "reservoir", "reservoir", "PHS", "battery"]
    ).sum()

    storage["max_hours"] = storage_capacity.unstack().reindex(
        pd.MultiIndex.from_arrays(
            [storage.bus, storage.carrier]  
        )
    ).div(
        storage.p_nom.values
    ).values

    map_storage = pd.Series( ["battery inverter", "PHS", "hydro"], ["battery", "PHS", "reservoir"])

    storage["efficiency"] = technology_data.loc[:, "efficiency", :].reindex(storage.carrier.map(map_storage)).value.values

    n.madd(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows,
        invest_status = "existing"
    )

In [8]:
def prepare_commodity_prices(commodity_prices):

    commodity_prices = commodity_prices_raw.iloc[:, 2:].groupby(commodity_prices.index).mean()
    commodity_prices = commodity_prices.groupby(["co2", "CCGT", "coal", "oil", "hydrogen", "oil", "lignite", "nuclear", "oil"]).mean()
       
    converter = pd.DataFrame(3.6 , commodity_prices.index, commodity_prices.columns)
    converter["CO2"] = 3.6/1000
    converter.loc["co2"] = 1
    
    commodity_prices = converter*commodity_prices

    to_add = commodity_prices.reindex(["CCGT", "CCGT", "CCGT"])
    to_add.index = ["OCGT", "biomass", "other"]
    to_add.loc[["other"]] = 0
    to_add.loc[["biomass"]] = biomass_price
    to_add.loc[["biomass"], "CO2"] = 0
    commodity_prices = pd.concat([commodity_prices, to_add])
    
    return commodity_prices

In [9]:
def add_existing_dispatchables(legacy):
    
    legacy["efficiency"] = properties["Standard efficiency in NCV terms"].loc[:, "existing"].reindex(legacy.carrier).values

    legacy["start_up_cost"] = properties["Start-up fix cost (e.g. wear) warm start"].loc[:, "existing"].reindex(legacy.carrier, level=1).fillna(0).values
    legacy["ramp_limit_up"] = properties["Ramp up rate % of max output power / min"].loc[:, "existing"].reindex(legacy.carrier).values*60
    legacy["ramp_limit_down"] = properties["Ramp down rate % of max output power / min"].loc[:, "existing"].reindex(legacy.carrier).values*60
    legacy["p_min_pu"] = properties["Minimum stable generation (% of max power)"].loc[:, "existing"].reindex(legacy.carrier).values
    legacy["min_up_time"] = properties["Min Time on"].loc[:, "existing"].reindex(legacy.carrier).values
    legacy["min_down_time"] = properties["Min Time off"].loc[:, "existing"].reindex(legacy.carrier).values

    legacy["marginal_cost"] = commodity_prices.reindex(legacy.carrier)[base_year].div(
        legacy.efficiency.values,
    ).add(
        (
            commodity_prices.reindex(
                legacy.carrier)["CO2"]
            .multiply(
                commodity_prices.loc["co2", base_year]
            )
        )
    ).add(
        properties.loc[:, "existing",:].reindex(legacy.carrier)["Variable O&M cost"]
    ).values

    legacy = legacy.loc[(legacy.entry <= year) & (legacy.exit > year)]
    
    n.madd(
        "Generator",
        legacy.index,
        **legacy,
        invest_status = "existing"
    )

In [10]:
def add_renewables():
    
    res = capacity.loc[["onwind","offwind", "solar", "CSP", "ROR"]].unstack().copy()
    res = res.reset_index()
    res.columns = ["bus", "carrier", "p_nom"]
    res.index = res.bus + " " + res.carrier

    res = res[res.p_nom>0]

    vre = ["onwind", "offwind", "solar", "CSP"]

    base_year_res = int(plant_sheets[plant_sheets >= year].subtract(year).idxmin()[2:])
    
    p_max_pu = pd.concat(
        [pd.read_hdf("resources/res_profile.h5", tech).loc[:, str(climate_year), str(base_year_res), :] for tech in vre],
        axis=1
    )

    p_max_pu.columns = vre
    p_max_pu = p_max_pu.unstack(0).stack(0)
    p_max_pu.index = [" ".join(i) for i in p_max_pu.index]
    p_max_pu.columns = snapshots
    p_max_pu = p_max_pu.T

    p_max_pu = pd.concat(
        [p_max_pu, inflows[res.filter(like="ROR", axis=0).index].div(res.filter(like="ROR", axis=0).p_nom)],
        axis=1
    )

    n.madd(
        "Generator",
        res.index,
        **res,
        p_max_pu = p_max_pu[res.index],
        invest_status="policy"
    )

In [11]:
def group_luxembourg(demand, links):
    
    demand_grouper = pd.Series(demand.columns, demand.columns)
    demand_grouper.loc[demand_grouper.index.str[:2] == "LU"] = "LUG1"
    demand = demand.groupby(demand_grouper, axis=1).sum()

    links.loc[links.bus0.str[:2] == "LU", "bus0"] = "LUG1"
    links.loc[links.bus1.str[:2] == "LU", "bus1"] = "LUG1"
    
    return demand, links

In [12]:
def add_dsr():
    
    p_nom_dsr.columns = ["band " + i.split(" ")[2] for i in p_nom_dsr.columns]

    marginal_cost_dsr.columns = ["band " + i.split(" ")[8] for i in marginal_cost_dsr.columns]

    dsr = pd.concat([p_nom_dsr.stack(), marginal_cost_dsr.stack()],axis=1)

    dsr = dsr.loc[:, base_year, :]

    dsr = dsr.set_index((i[0] + " dsr " + i[1] for i in dsr.index), append=True).reset_index([0,1]).drop("level_1",axis=1)

    dsr.columns = ["bus", "p_nom", "marginal_cost"]
    
    dsr["carrier"] = "dsr"

    n.madd(
        "Generator",
        dsr.index,
        **dsr,
        invest_status = "existing"
    )

In [13]:
def add_dispatchable_investment_options():

    zones_for_investment = capacity.sum()[capacity.loc[dispatchable].sum() >0 ].index # no investment in offshores zones etc.


    new_dispatchables = pd.DataFrame(
        0.01, 
        index= pd.MultiIndex.from_product(
            [zones_for_investment,technologies_for_investment, range(years[0], year+1)], 
            names=["bus", "carrier", "entry"]), 
        columns=["p_nom"]
    )


    new_dispatchables.reset_index(inplace=True)

    new_dispatchables.index = new_dispatchables.bus + " " + new_dispatchables.carrier + " new " + new_dispatchables.entry.astype(str)

    new_dispatchables["efficiency"] = properties.loc[:, "new", :]["Standard efficiency in NCV terms"].reindex(new_dispatchables.carrier).values

    new_dispatchables["start_up_cost"] = properties["Start-up fix cost (e.g. wear) warm start"].loc[:, "new"].reindex(new_dispatchables.carrier, level=1).fillna(0).values

    new_dispatchables["ramp_limit_up"] = properties["Ramp up rate % of max output power / min"].loc[:, "new"].reindex(new_dispatchables.carrier).values*60
    new_dispatchables["ramp_limit_down"] = properties["Ramp down rate % of max output power / min"].loc[:, "new"].reindex(new_dispatchables.carrier).values*60
    new_dispatchables["p_min_pu"] = properties["Minimum stable generation (% of max power)"].loc[:, "new"].reindex(new_dispatchables.carrier).values
    new_dispatchables["min_up_time"] = properties["Min Time on"].loc[:, "new"].reindex(new_dispatchables.carrier).values
    new_dispatchables["min_down_time"] = properties["Min Time off"].loc[:, "new"].reindex(new_dispatchables.carrier).values

    new_dispatchables["marginal_cost"] = commodity_prices.reindex(new_dispatchables.carrier)[base_year].div(
            new_dispatchables.efficiency.values,
        ).add(
            (
                commodity_prices.reindex(
                    new_dispatchables.carrier)["CO2"]
                .multiply(
                    commodity_prices.loc["co2", base_year]
                )
            )
        ).add(
            properties.loc[:, "existing",:].reindex(new_dispatchables.carrier)["Variable O&M cost"]
        ).values

    n.madd(
        "Generator",
        new_dispatchables.index,
        **new_dispatchables,
        invest_status = "new"
    )

In [14]:
def set_investment_bounds():
    
    n.generators.loc[n.generators.invest_status == "existing", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "existing", "p_nom"].clip(0.01)
    n.generators.loc[n.generators.invest_status == "existing", "p_nom"] = n.generators.loc[n.generators.invest_status == "existing", "p_nom"].clip(0.01)
    n.generators.p_nom_min = 0.01
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_min"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]

In [15]:
commodity_prices_raw = pd.read_excel("data/Fuel and CO2 prices_ERAA2023.xlsx", index_col = 0)

In [16]:
year = 2029

In [17]:
climate_year = 1990

In [18]:
years = range(2025, 2034)

In [19]:
save_path = "resources/networks/base/cy" + str(climate_year) + "_ty" + str(year) + ".nc"

In [20]:
biomass_price = 7 # make adjustable in config

In [21]:
dispatchable = ["CCGT", "OCGT", 'biomass', 'coal', 'lignite', 'nuclear','oil',  'other', ]

In [22]:
technologies_for_investment = ["OCGT", "CCGT"]

In [23]:
inflows_raw = pd.read_hdf("resources/inflow.h5", "inflow")

In [24]:
excel_file = pd.ExcelFile("data/pemmdb.xlsx")

In [25]:
snapshots = pd.date_range(start="2010-01-01", freq="h", periods=8760)

In [26]:
plant_sheets = [i for i in excel_file.sheet_names if "TY" in i]
plant_sheets = pd.Series([int(i[2:]) for i in plant_sheets], plant_sheets )

In [27]:
sheet = plant_sheets[plant_sheets <= year].subtract(year).idxmax()

In [28]:
base_year = int(sheet[2:])

In [29]:
distributed_resources = ["onwind", "offwind", "CSP","solar", "battery"]

In [30]:
demand = pd.read_hdf("resources/demand.h5")
demand = demand.loc[:, climate_year, :, str(base_year), :].unstack(1)
demand.index = snapshots
demand.drop("TR00",axis=1, inplace=True)

In [31]:
links = pd.read_hdf("resources/ntcs.h5", "p_nom")
links_p_max_pu = pd.read_hdf("resources/ntcs.h5", "p_max_pu")
links = links[base_year].unstack(1)
links_p_max_pu = links_p_max_pu[base_year].unstack(2)
links_p_max_pu.index = snapshots

In [32]:
links.dropna(inplace=True)
links_p_max_pu.dropna(axis=1, inplace=True)

In [33]:
links["carrier"] = [i[-2:] for i in links.index]

In [34]:
demand, links = group_luxembourg(demand, links)

In [35]:
commodity_prices = prepare_commodity_prices(commodity_prices_raw)

In [36]:
dispatchable_plants = pd.read_hdf("resources/dispatchable_plants.h5")

In [37]:
technology_data = pd.read_csv("../technology-data/outputs/costs_2025.csv", index_col=[0,1])

In [38]:
n = pypsa.Network()
n.set_snapshots(snapshots)

In [39]:
capacity = build_capacity_table()

In [40]:
p_nom_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, :8]
marginal_cost_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, 8:16]

In [41]:
buses = (
    capacity.sum()[capacity.sum() >0].index
    .union(demand.columns)
    .union(links.bus0.unique())
    .union(links.bus1.unique())
)

In [42]:
n.madd(
    "Bus", 
    buses, 
    carrier = "electricity", 
    country = buses.str[:2]
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BEOF', 'BG00', 'CH00', 'CY00', 'CZ00',
       'DE00', 'DEKF', 'DKBH', 'DKE1', 'DKKF', 'DKW1', 'EE00', 'ES00', 'FI00',
       'FR00', 'GR00', 'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS',
       'ITN1', 'ITS1', 'ITSA', 'ITSI', 'ITVI', 'LT00', 'LUG1', 'LV00', 'ME00',
       'MK00', 'MT00', 'NL00', 'NLLL', 'NOM1', 'NON1', 'NOS0', 'PL00', 'PLE0',
       'PLI0', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04', 'SI00',
       'SK00', 'UK00', 'UKNI'],
      dtype='object')

In [43]:
n.madd(
    "Load", 
    demand.columns,
    bus=demand.columns,
    p_set = demand
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BG00', 'CH00', 'CY00', 'CZ00', 'DE00',
       'DKE1', 'DKW1', 'EE00', 'ES00', 'FI00', 'FR00', 'GR00', 'GR03', 'HR00',
       'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI',
       'LT00', 'LUG1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00', 'NOM1', 'NON1',
       'NOS0', 'PL00', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04',
       'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object', name='zone')

In [44]:
inflows = build_inflows(inflows_raw)

In [45]:
add_existing_storage()

In [56]:
dispatchable_plants = dispatchable_plants.query("(entry <= @base_year) & exit > @base_year")

In [58]:
commodity_prices.reindex(dispatchable_plants.carrier)[base_year].div(
            dispatchable_plants.efficiency.values,
        ).add(
            (
                commodity_prices.reindex(
                    dispatchable_plants.carrier)["CO2"]
                .multiply(
                    commodity_prices.loc["co2", base_year]
                )
            )
        ).add(
            properties.loc[:, "existing",:].reindex(dispatchable_plants.carrier)["Variable O&M cost"]
        ).values

NameError: name 'properties' is not defined

In [46]:
dispatchable_plants

,bus,p_nom,carrier,entry,exit,efficiency,start_up_cost,ramp_limit_up,ramp_limit_down,p_min_pu,min_up_time,min_down_time,p_nom_max,p_nom_min
AL00 CCGT exit 2034,AL00,86.649501,CCGT,2025,2034.0,0.521667,39.025,2.0,3.0,0.433333,2.666667,2.666667,86.649501,0.01
AL00 CCGT entry 2028,AL00,173.299002,CCGT,2028,2034.0,0.521667,39.025,2.0,3.0,0.433333,2.666667,2.666667,173.299002,0.01
AT00 CCGT exit 2028,AT00,883.000000,CCGT,2025,2028.0,0.521667,39.025,2.0,3.0,0.433333,2.666667,2.666667,883.000000,0.01
AT00 CCGT exit 2030,AT00,2.830000,CCGT,2025,2030.0,0.521667,39.025,2.0,3.0,0.433333,2.666667,2.666667,2.830000,0.01
AT00 CCGT exit 2033,AT00,1225.030000,CCGT,2025,2033.0,0.521667,39.025,2.0,3.0,0.433333,2.666667,2.666667,1225.030000,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UKNI CCGT new 2029,UKNI,0.010000,CCGT,2029,NaN,0.600000,25.060,2.4,3.0,0.400000,2.000000,2.000000,inf,0.01
UKNI CCGT new 2030,UKNI,0.010000,CCGT,2030,NaN,0.600000,25.060,2.4,3.0,0.400000,2.000000,2.000000,inf,0.01
UKNI CCGT new 2031,UKNI,0.010000,CCGT,2031,NaN,0.600000,25.060,2.4,3.0,0.400000,2.000000,2.000000,inf,0.01
UKNI CCGT new 2032,UKNI,0.010000,CCGT,2032,NaN,0.600000,25.060,2.4,3.0,0.400000,2.000000,2.000000,inf,0.01


In [281]:
add_renewables()

In [282]:
add_dsr()

In [284]:
set_investment_bounds()

In [285]:
n.generators

,bus,p_nom,carrier,entry,exit,efficiency,start_up_cost,ramp_limit_up,ramp_limit_down,p_min_pu,...,capital_cost,committable,shut_down_cost,stand_by_cost,up_time_before,down_time_before,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
AL00 CCGT exit 2034,AL00,88.225784,CCGT,2025.0,2034.0,0.521667,39.025,2.0,3.0,0.433333,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AL00 CCGT entry 2028,AL00,176.451569,CCGT,2028.0,2034.0,0.521667,39.025,2.0,3.0,0.433333,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AT00 CCGT exit 2030,AT00,2.499548,CCGT,2025.0,2030.0,0.521667,39.025,2.0,3.0,0.433333,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AT00 CCGT exit 2033,AT00,1081.986127,CCGT,2025.0,2033.0,0.521667,39.025,2.0,3.0,0.433333,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AT00 CCGT exit 2034,AT00,1797.607475,CCGT,2025.0,2034.0,0.521667,39.025,2.0,3.0,0.433333,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UKNI CCGT new 2025,UKNI,0.010000,CCGT,2025.0,NaN,0.600000,25.060,2.4,3.0,0.400000,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
UKNI CCGT new 2026,UKNI,0.010000,CCGT,2026.0,NaN,0.600000,25.060,2.4,3.0,0.400000,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
UKNI CCGT new 2027,UKNI,0.010000,CCGT,2027.0,NaN,0.600000,25.060,2.4,3.0,0.400000,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0


In [286]:
n.madd(
    "Link",
    links.index,
    bus0 = links.bus0,
    bus1 = links.bus1,
    p_nom = links.p_nom,
    p_max_pu = links_p_max_pu,
    carrier = links.carrier
)

Index(['AL00-GR00-AC', 'AL00-ME00-AC', 'AL00-MK00-AC', 'AL00-RS00-AC',
       'AT00-CH00-AC', 'AT00-CZ00-AC', 'AT00-DE00-AC', 'AT00-HU00-AC',
       'AT00-ITN1-AC', 'AT00-SI00-AC',
       ...
       'UK00-BE00-DC', 'UK00-DE00-DC', 'UK00-DKW1-DC', 'UK00-FR00-DC',
       'UK00-IE00-DC', 'UK00-NL00-DC', 'UK00-NOS0-DC', 'UK00-UKNI-DC',
       'UKNI-IE00-AC', 'UKNI-UK00-DC'],
      dtype='object', length=212)

In [287]:
n.madd(
    "Generator",
    n.buses.index + " load-shedding",
    bus = n.buses.index,
    p_nom=1e5,
    marginal_cost = 10e3,
)

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BEOF load-shedding', 'BG00 load-shedding',
       'CH00 load-shedding', 'CY00 load-shedding', 'CZ00 load-shedding',
       'DE00 load-shedding', 'DEKF load-shedding', 'DKBH load-shedding',
       'DKE1 load-shedding', 'DKKF load-shedding', 'DKW1 load-shedding',
       'EE00 load-shedding', 'ES00 load-shedding', 'FI00 load-shedding',
       'FR00 load-shedding', 'GR00 load-shedding', 'GR03 load-shedding',
       'HR00 load-shedding', 'HU00 load-shedding', 'IE00 load-shedding',
       'ITCA load-shedding', 'ITCN load-shedding', 'ITCS load-shedding',
       'ITN1 load-shedding', 'ITS1 load-shedding', 'ITSA load-shedding',
       'ITSI load-shedding', 'ITVI load-shedding', 'LT00 load-shedding',
       'LUG1 load-shedding', 'LV00 load-shedding', 'ME00 load-shedding',
       'MK00 load-shedding', 'MT00 load-shedding', 'NL00 load-shedding',
       'NLLL load-shedding', 'NOM1 load-shedding', 

In [291]:
n.generators.loc[(n.generators.bus == "CY00") & (n.generators.carrier == "CCGT"), "p_min_pu"] = 0

In [114]:
dirname = os.path.dirname(save_path)  

if not os.path.exists(dirname):
    os.makedirs(dirname)

In [56]:
n.export_to_netcdf(save_path)

INFO:pypsa.io:Exported network cy1990_ty2029.nc has buses, storage_units, links, loads, generators


<xarray.Dataset>
Dimensions:                       (snapshots: 8760, investment_periods: 0,
                                   buses_i: 57, storage_units_i: 89,
                                   storage_units_t_inflow_i: 33, links_i: 212,
                                   links_t_p_max_pu_i: 212, loads_i: 49,
                                   loads_t_p_set_i: 49, generators_i: 917,
                                   generators_t_p_max_pu_i: 170)
Coordinates:
  * snapshots                     (snapshots) int64 0 1 2 3 ... 8757 8758 8759
  * investment_periods            (investment_periods) int64 
  * buses_i                       (buses_i) object 'AL00' 'AT00' ... 'UKNI'
  * storage_units_i               (storage_units_i) object 'AL00 reservoir' ....
  * storage_units_t_inflow_i      (storage_units_t_inflow_i) object 'AL00 res...
  * links_i                       (links_i) object 'AL00-GR00-AC' ... 'UKNI-U...
  * links_t_p_max_pu_i            (links_t_p_max_pu_i) object 'AL00-GR00-AC' ...
  * loads_i                       (loads_i) object 'AL00' 'AT00' ... 'UKNI'
  * loads_t_p_set_i               (loads_t_p_set_i) object 'AL00' ... 'UKNI'
  * generators_i                  (generators_i) object 'AL00 CCGT' ... 'UKNI...
  * generators_t_p_max_pu_i       (generators_t_p_max_pu_i) object 'AL00 onwi...
Data variables: (12/38)
    snapshots_snapshot            (snapshots) datetime64[ns] 2010-01-01 ... 2...
    snapshots_objective           (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_stores              (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_generators          (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    investment_periods_objective  (investment_periods) object 
    investment_periods_years      (investment_periods) object 
    ...                            ...
    generators_marginal_cost      (generators_i) float64 83.75 104.8 ... 1e+04
    generators_invest_status      (generators_i) object 'existing' ... nan
    generators_p_nom_min          (generators_i) float64 0.01 0.01 ... 0.0 0.0
    generators_p_nom_max          (generators_i) float64 264.7 35.32 ... inf inf
    generators_invest_year        (generators_i) float64 nan nan nan ... nan nan
    generators_t_p_max_pu         (snapshots, generators_t_p_max_pu_i) float64 ...
Attributes:
    network_name:           
    network_pypsa_version:  0.26.2
    network_srid:           4326
    crs:                    {"_crs": "GEOGCRS[\"WGS 84\",ENSEMBLE[\"World Geo...
    meta:                   {}

In [57]:
n.optimize(snapshots=n.snapshots[:24], solver_name="highs")

RuntimeError: No solver installed.

In [ ]:
n = pypsa.Network("resources/networks/0/")